In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.linalg import Vectors
from sklearn.datasets import make_classification
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np

spark = (SparkSession.builder
         .appName("Gaussian Naive Bayes")
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/15 22:51:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# load data
X, y = make_classification(
    n_samples=1000,
    n_features=3,
    n_informative=3,
    n_redundant=0,
    n_clusters_per_class=1,
    n_classes=3,
    weights=[0.3, 0.3, 0.4],
    flip_y=0.1,
    random_state=87
)
X = np.abs(X)

data = [(Vectors.dense(x.tolist()), float(label)) for x, label in zip(X, y)]
spark_df = spark.createDataFrame(data, ["features", "label"])

# split data
train, test = spark_df.randomSplit([0.8, 0.2], seed=87)

# train model
nb = NaiveBayes(
    featuresCol='features',
    labelCol='label',
    modelType='gaussian'
)
nb_model = nb.fit(train)

# predict
predictions = nb_model.transform(test)

# evaluate
evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='accuracy'
)
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='weightedPrecision'
)
precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='weightedRecall'
)
recall = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='f1'
)
f1 = evaluator.evaluate(predictions)

print("=" * 50)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)
print("=" * 50)

Accuracy: 0.4205607476635514
Precision: 0.4416810276020201
Recall: 0.42056074766355134
F1: 0.4096317064313373
